# Machine Learning Quick Start

In [ ]:
import pyspark
import sys

In [ ]:
import pyspark.sql.functions as fn

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [ ]:
# Check spark app name
spark.sparkContext.appName

In [ ]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)

In [ ]:
# print runtime versions
# Python version
sys.version

In [ ]:
# Spark version
spark.version

### Exploring Data

In [ ]:
# load iris.csv into Spark dataframe
df = spark.read.csv('data/iris.csv', header=True, inferSchema=True)

In [ ]:
# First 5 rows of Iris dataset
df.show(5)

In [ ]:
df.printSchema()

In [ ]:
df.describe().show()

In [ ]:
# number of records for each species available in the dataset
df.groupBy('species').count().show(10,False)

### Feature Engineering

In [ ]:
# display all column names
df.columns

In [ ]:
# vectorize all numerical columns into a single feature column
feature_cols = df.columns[:-1]
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
df = assembler.transform(df)

In [ ]:
# convert text labels into indices
data = df.select(['features', 'species'])
label_indexer = StringIndexer(inputCol='species', outputCol='label').fit(data)
data = label_indexer.transform(data)

In [ ]:
# only select the features and label column
data = data.select(['features', 'label'])

In [ ]:
# Reading for machine learning
data.show(10)

In [ ]:
data.select(['label']).distinct().show()

### Split Data - Train & Test sets

In [ ]:
# use Logistic Regression to train on the training set
train, test = data.randomSplit([0.70, 0.30], seed=42)

### Build Logistic Regression Model

In [ ]:
# change regularization rate and you will likely get a different accuracy.
reg = 0.01

In [ ]:
lr = LogisticRegression(regParam=reg)
model = lr.fit(train)

In [ ]:
# predict on the test set
prediction = model.transform(test)

In [ ]:
# print prediction
prediction.show(10)

### Evaluate Model

In [ ]:
# evaluate the accuracy of the model using the test set
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [ ]:
accuracy = evaluator.evaluate(prediction)

In [ ]:
# print accuracy 
accuracy